#### 1. zookeeper有哪些使用场景?
* **分布式协调 (zk做回调)**  
  分布式服务中, 系统A发送一个请求到mq, 系统B从mq里接收消息后处理. 那系统A如何知道系统B处理完毕了呢? 一个可行的做法是:   
  系统A发送完消息后, 注册一个zk节点; 系统B收到消息并处理后修改zk节点上的值, 此时系统A会立刻收到通知  
  
* **分布式锁**  
  假设有2个系统同时接收到了一个请求, 但只有其中的一个系统可以执行该请求, 如何进行排它锁呢?   
  收到请求的系统可以在zk上创建一个临时节点, (创建临时节点主要是考虑到避免获得锁的系统因为宕机无法释放锁而产生死锁)
    * 先收到请求的系统会创建成功, 表示获得了锁; 
    * 后收到请求的系统会创建失败, 表示锁获取失败, 此后在这个zk临时节点上建立一个监听器, 一旦发现该节点被删除(有某个系统释放了锁), 就再次请求创建节点(获取锁). 如果是redis实现分布式锁, 获取锁失败的系统要每隔1s取轮训`setnx key 值 `
  
* **元数据配置**   
  因为元数据要求高可用和一致性, 所以用zk实现这些特点   
  
* **HA选主**   
  可以动态选主, 实现准备master之间的自动切换  

### 2. [重要]分布式锁是啥?  

#### 2.1 redis中有什么分布式锁实现 [redis](http://mp.weixin.qq.com/s?__biz=MzIxMjE5MTE1Nw==&mid=2653194065&idx=1&sn=1baa162e40d48ce9b44ea5c4b2c71ad7&chksm=8c99f58bbbee7c9d5b5725da5ee38fe0f89d7a816f3414806785aea0fe5ae766769600d3e982&scene=21#wechat_redirect)
1. 最普通的实现方法  
  在redis中创建一个key表示锁`set my:lock 随机值 NX PX 30000`: 创建1个my:lock, 如果不存在就创建成功, 该key在30秒后失效.   
  只有创建成功的系统才表示获取锁成功, 该系统在处理完业务逻辑后, 使用lua脚本删除`my:lock`
    1. 为啥用随机值?   
      因为如果某个客户端获取了锁, 但是因为执行时间很长, 锁可能在redis中已过期, 于是其它系统获取锁; 如果不加判断就删除锁可能会误删其他系统获得的锁
    2. 该做法的一个问题是: 如果是采用一主多从的架构, master宕机后, 新的slave提升成master时可能并不包含锁, 导致其他系统也获得了锁;
    ```lua
        if redis.call(get,"my:lock"=="预期值") then
          return redis.call("del","my:lock")
        else
          return 0
        end
    ```
2. RedLock算法  
 该算法假设在redis-cluster环境中, 假设有5个master, 通过如下步骤获得一个锁  
    1. 获取当前时间戳(单位毫秒)
    2. 轮询在每个master上创建锁
    3. 如果超过一半的master都建锁成功, 则该系统获得了锁;  
      否则, 依次删除建立好的锁
    4. 只要获取锁失败, 系统就要不断`轮训->建立->判断成功建立的是否大于一半->删除` 
    
#### 2.2 zookeeper实现分布式锁
基于zookeeper的临时顺序节点实现分布式锁:  
1. **获取锁**
    * 首先, 在zookeeper中常见一个永久节点ParentLock, 当客户端需要获得锁时, 就在ParentLock下面创建临时顺序节点lock
    * 然后, 客户端查询ParentLock下面的所有子节点并排序, 判断自己所创建的节点是不是列表内的第一个节点:
        * 如果是, 则该客户端成功获取锁
        * 如果不是, 则对列表中紧挨它的前面一个临时顺序节点注册watcher, 且宣布本次抢占锁失败.  
        ```
        water的目的是, 当前面的临时顺序节点销毁后, 进行监听的客户端会受到回调消息, 证明自己对锁抢占成功
        ```
2. **释放锁**
    * 最后, 使用完锁的客户端要释放锁. 可以有客户端主动调用删除的API进行删除, 也可以是客户端运行期间崩溃, 导致和zookeeper的seesion过期而自动删除节点. 节点删除意味着锁释放完毕

#### 3. zookeeper有哪几种节点
* PERSISTENT 持久节点  
```
创建之后一直存在，除非执行删除操作，否则即使创建节点的客户端会话失效也该节点也仍然会存在。 不能创建同名节点
```
* PERSISTENT_SEQUENTIAL 持久顺序节点
```
跟持久一样，客户端断开连接后仍然存在， 不同的是节点名会被自动加上一个序号后缀, 这个序号自从server建立起后就不断增加
```
<img src="img/zk1.png" width="99%">
* EPHEMERAL 临时节点
```
创建客户端会话失效（注意是会话失效，不是连接断了），节点也就没了。不能建子节点。
```
* EPHEMERAL_SEQUENTIAL 临时顺序节点
```
基本特性同临时节点，增加了顺序属性，节点名后边会追加一个由父节点维护的自增整型数字。
```

#### 4. Zookeeper 的通知机制是什么？
Zookeeper 允许客户端向服务端的某个 znode 注册一个 Watcher 进行监听，当被监听的节点数据发生变化就会触发Watcher ，服务端会向指定客户端发送一个事件通知, 然后客户端根据 Watcher 通知状态和事件类型做出业务上的改变。整个流程如下：
* 第一步，客户端注册 Watcher
* 第二步，服务端处理 Watcher
* 第三步，客户端回调 Watcher

#### 5. Zookeeper watcher的特点是什么   
* **一次性触发**:  
  一个 Watcher 只能对接点进行一次监听， 一旦被监听的节点触发了事件, 这个watcher就会失效 。这样的设计有效的减轻了服务端的压力，不然对于更新非常频繁的节点，服务端会不断的向客户端发送事件通知. 如果要持续坚挺, 需要在watcher的回调里面重新对节点绑定监听
    ```
    如果我们使用 Apache Curator 作为操作 Zookeeper 的客户端，它可以帮我们自动透明的实现持续的 watch 操作，非常方便。
    ```
* **客户端串行执行**   
   客户端 Watcher 回调的过程是一个串行同步的过程。
* **轻量级 Watch 机制**  
    * Watcher 通知非常简单，只会告诉客户端发生了事件，而不会说明事件的具体内容
    * 客户端向服务端注册 Watcher 的时候，并不会把客户端真实的 Watcher 对象实体传递到服务端，仅仅是在客户端请求中使用boolean 类型属性进行了标记。    
* **Watcher event 异步发送 Watcher 的通知事件**     
   从 Server 发送到Client 是异步的，这就存在一个问题，不同的客户端和服务器之间通过Socket 进行通信，由于网络延迟或其他因素导致客户端在不通的时刻监听到事件，由于 Zookeeper 本身提供了 ordering guarantee ，即客户端监听事件后，才会感知它所监视 znode 发生了变化。所以我们使用 Zookeeper 不能期望能够监控到节点每次的变化。Zookeeper 只能保证最终的一致性，而无法保证强一致性。
* **可以注册 Watcher 的操作**：  
   ```
   getData、exists、getChildren
   ````
* **可以触发 Watcher 的操作**：
   ```
   create、delete、setData
   ```
* **当一个 Client 连接到一个新的服务器上时，watch 将会被以任意会话事件触发**  
   当与一个服务器失去连接的时候，是无法接收到 watch 的。而当 Client 重新连接时，如果需要的话，所有先前注册过的watch ，都会被重新注册。通常这是完全透明的。只有在一个特殊情况下，watch 可能会丢失：对于一个未创建的 znode 的 exists watch ，如果在客户端断开连接期间被创建了，并且随后在客户端连接上之前又删除了，这种情况下，这个 watch 事件可能会被丢失。
   
   
#### 6. Zookeeper 的会话管理是怎么样的？
* ZooKeeper的每个客户端都维护一组服务端信息，在创建连接时由应用指定，客户端随机选择一个服务端进行连接
* 连接成功后，服务端为每个连接分配一个唯一标识, 作为SessionId
* 客户端会周期性的向服务端发送PING请求来保持连接，当客户端检测到与服务端断开连接后，客户端将自动选择服务端列表中的另一个服务端进行重连。
* 如果服务端一定时间没接收到客户端的ping请求, 就认为该客户端已下线  

### 7. Zookeeper的一致性
zookeeper为了保证每个Server之间的同步, 实现了ZAB(zookeeper atomic broadcast)协议进行选主和同步, 分别对应ZAB的两种模式: 
* 恢复模式(选主)和
* 广播模式(同步).    

首先, ZAB使用FastLeaderElection的算法进行选主, 它将服务器定义成4种状态:
* LOOKIN
```
寻找Leader状态。当服务器处于该状态时，它会认为当前集群中没有Leader，因此需要进入Leader选举状态。
```
* FOLLOWING
```
跟随者状态。表明当前服务器角色是Follower
```
* LEADING
```
领导者状态。表明当前服务器角色是Leader
```
* OBSERVING
```
观察者状态。表明当前服务器角色是Observer。
```

然后, 选主会发生在2个的阶段: `服务器初始化时进行leader选举` 和 ``
####  **服务器启动时期的Leader选举**, 选举过程如下:  
1. **每个节点自动发起投票**  
  投票的格式主要包括`SID`和`ZID`, 分别表示推举的服务器的id和推举的服务器的事务ID. 每个节点都会推举自己为leader, 因此SID为自己. 而初始时事务ID为0, 所以如果有2台节点, server1和server2, server1的投票为(1,0), server2的投票为(2,0); 投票会发送到所有其他节点上. 投票的具体数据结构为:
    * `id`：被推举的Leader的SID。
    * `zxid`：被推举的Leader事务ID。
    * `electionEpoch`：逻辑时钟，用来判断多个投票是否在同一轮选举周期中，该值在服务端是一个自增序列，每次进入新一轮的投票后，都会对该值进行加1操作。只有处于同一轮几时钟下的投票才有比较的必要   
    * `peerEpoch`：被推举的Leader的epoch。
    * `state`：当前服务器的状态。
2. **接收各个服务器的投票**  
   服务器的节点在收到投票后, 先检查投票的有效性: 包括是否是本轮投票, 投票来自的节点是否是LOOKING状态的节点, 最重要的是比较受到的投票和本地投票的轮次(逻辑时钟)
    * 如果**外部投票的选举轮次大于内部投票**。说明节点自身的投票已经失效, 所以立刻更改自己的逻辑时钟为外部投票的逻辑时钟, 并放弃所有已收到的投票(只有逻辑时钟相同的外部投票才能加入节点的recieve列表中, 若自身时钟过期, 说明所有收到的投票时钟都过期了), 并用自身的ZID和SID与外部投票进行比较(比较规则和第三点一样), 产生新的投票后发送给急群众所有的其他节点
    * 如果**外部投票的选举轮次小于内部投票**。若服务器接收的外选票的选举轮次落后于自身的选举轮次，那么Zookeeper就会直接忽略该外部投票，不做任何处理
    * 如果**外部投票的选举轮次等于内部投票**。此时可以开始进行第3步选票PK。
3. **更改自己的本地投票**  
   每个节点在收到来自其他节点的投票, 且验证有效后, 会将其和自己的本地投票做比较, 从而产生新的本地投票(第二次投票). 比较规则如下
    * **先比较ZID**, 选择ZID数值大的作为自己的本地投票  
    * **如果ZID相同, 再比较SID**, 选择SID大的作为自己的本地投票
4. **统计投票**  
   每次投票后, 节点会统计投票信息, 判断是否有过半的机器推举了同一台机器作为leader:  
    * 如果有: leader已经选举出来
    * 如果没有: 重新将更新后的投票发送给集群的其他节点, 进行新一轮的投票, 直到选择出了leader  
5. **修改节点状态**  
   一旦确定了由过半机器投给了同一台节点, 判断哪个节点是否是自己, 是的话自己就成为了leader, 状态更改为LEADING; 不是的话自己是FOLLOWER, 更改状态为FOLLOWING

#### **服务器运行时期的leader选举**
在Zookeeper运行期间，一旦Leader服务器挂了，那么整个集群将暂停对外服务，进入新一轮Leader选举，其过程和启动时期的Leader选举过程基本一致。  
假设正在运行的有Server1、Server2、Server3三台服务器，当前Leader是Server2，若某一时刻Leader挂了，此时便开始Leader选举。选举过程如下
1. **变更状态**。Leader挂后，余下的非Observer服务器都会将自己的服务器状态变更为LOOKING，然后开始进入Leader选举过程。
2. **每个Server会发出一个投票**。在运行期间，每个服务器上的ZXID可能不同，此时假定Server1的ZXID为123，Server3的ZXID为122；在第一轮投票中，Server1和Server3都会投自己，产生投票(1, 123)，(3, 122)，然后各自将投票发送给集群中所有机器。
3. **接收来自各个服务器的投票**。与启动时过程相同。
4. **处理投票**。与启动时过程相同，此时，Server1将会成为Leader。
5. **统计投票**。与启动时过程相同。
6. **改变服务器的状态**。与启动时过程相同。

当服务器选主成功后, 就会进入同步阶段  


#### 8. zookeeper是否会产生脑裂?  
redis怎么实现分布式锁 (上面有链接)   
raft一致性中文翻译, 对比一下[raft](https://blog.csdn.net/nirendao/article/details/84963328)